# A* Algorithm

## Code section
After running the following cells, go to Run section to start the program

In [66]:
#!sudo apt-get install python3.x-tk
%matplotlib tk

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.lines import Line2D

In [55]:
def init():
    global src_cid, obs_cid, key_cid, maze, fig,ax, grid, src, dest, xs, ys
    xs,ys = np.reshape(np.mgrid[1:l+1,1:b+1], (2,l*b))
    maze = np.zeros([l,b])

    fig, ax = plt.subplots()
    ax.set_title('A* Search Algorithm')
    grid = ax.scatter(xs,ys, s = 20, picker=True, pickradius=3, 
                   color='white', linewidth = 0.4, edgecolors ='grey')
    if src is None and dest is None:
        src_cid = fig.canvas.mpl_connect('pick_event', get_src)
    else:
        ax.scatter(src[0], src[1], 20, color='cyan')
        ax.scatter(dest[0], dest[1], 20, color='cyan')
        key_cid = fig.canvas.mpl_connect('key_press_event', on_key)
        if maze_grid is None:
            obs_cid = fig.canvas.mpl_connect('pick_event', add_obs)
        else:
            show_maze()
            obs_cid = None
    ##1. Remove spines
    #for spine in plt.gca().spines.values():
    #    spine.set_visible(False)
    ##2. Remove ticks
    plt.xticks([])
    plt.yticks([])
    fig.colorbar(matplotlib.cm.ScalarMappable(
        norm = matplotlib.colors.Normalize(vmin = 0, vmax = max(l,b) if allow_diagonal_movement else l+b),
                                              cmap = plt.get_cmap(cmap_name)), label = 'Heuristics')


    custom_lines = [Line2D([], [], color='cyan',marker = 'o', ms=4, lw = 0),
                    Line2D([], [], color='black',marker = obs_marker, ms=4, lw = 0),
                    Line2D([], [], color='lightgreen',marker = 'o', mew = 2, mec ='green', ms=5, lw = 0),
                    Line2D([], [], color='lightblue',marker = 'o', ms=5, lw = 0)]
    ax.legend(custom_lines, ['Source/Destination', 'Obstacle', 'Path', 'Unvisited node'],
              bbox_to_anchor=(0., -0.02, 1., 0.01), loc='upper left',
               ncol=2, mode="expand", borderaxespad=0.)#legend placement settings, tweak to change placement
    plt.show()

In [56]:
def get_src(event):
    global fig, ax, src, src_cid, dest_cid, xs, ys
    n = event.ind
    if not len(n):
        return

    ax.scatter(xs[n],ys[n],20,color='cyan')
    src = np.ravel([xs[n],ys[n]])
    print('Source: ', src)
    fig.canvas.mpl_disconnect(src_cid)
    dest_cid = fig.canvas.mpl_connect('pick_event', get_dest)
    fig.show()
    return True

In [57]:
def get_dest(event):
    global fig, ax, dest, key_cid, dest_cid, obs_cid, xs, ys
    n = event.ind
    if not len(n):
        return

    ax.scatter(xs[n],ys[n], 20,color='cyan')
    dest = np.ravel([xs[n],ys[n]])
    print('Destination: ', dest)
    fig.canvas.mpl_disconnect(dest_cid)
    if maze_grid is None:
        obs_cid = fig.canvas.mpl_connect('pick_event', add_obs)
    else:
        show_maze()
        obs_cid = None
    key_cid = fig.canvas.mpl_connect('key_press_event', on_key)
    fig.show()
    return True

In [58]:
def add_obs(event):
    global maze, ax, fig, xs, ys
    n = event.ind
    if not len(n):
        return
    if len(event.ind)>1:
        n = event.ind[0]
    i,j = np.ravel([xs[n],ys[n]])-1
    if maze[i,j]==1:
        ax.scatter(i+1,j+1, 20, color='white', linewidth = 0.4, edgecolors ='grey')
        maze[i,j] = 0
    elif maze[i,j]==0:
        ax.scatter(i+1,j+1, 20, color='black', marker = obs_marker)
        maze[i,j] = 1
    fig.show()
    return True

In [59]:
def on_key(event):
    global ax, fig, src, dest, obs_cid, key_cid, maze, show_iter, allow_diagonal_movement
    if event.key == 'enter':
        print('Initiating A* algorithm..')
        if obs_cid is not None:
            fig.canvas.mpl_disconnect(obs_cid)
            print(np.array2string(maze, separator=', '))
        path = astar(maze, src-1, dest-1, allow_diagonal_movement, show_iter)
        if path:
            show(path)
        fig.canvas.mpl_disconnect(key_cid)
    return True

In [60]:
def show_maze():
    global maze, fig, ax, maze_grid
    maze = maze_grid
    obs_x=[]
    obs_y=[]
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if(maze[i,j]==1):
                obs_x.append(i)
                obs_y.append(j)
    obs_x, obs_y = np.array([obs_x, obs_y])+1
    ax.scatter(obs_x, obs_y, 20, color='black', marker = obs_marker)
    fig.show()

In [61]:
def show(path):
    global ax, fig
    print('Path:')
    for node in path[:-1]:
        print(node, ' -->', end = ' ')
    print(path[-1])
    path_x, path_y = np.transpose(path)
    path_x, path_y = path_x+1, path_y+1
    ax.scatter(path_x, path_y, 30, color='lightgreen', linewidth = 2, edgecolors ='green')
    fig.show()
    plt.savefig(cmap_name+'success.png')

In [62]:
# Credit for this: Nicholas Swift
# as found at https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2
from warnings import warn
import heapq
from itertools import count

# a global
entry_id = count()
class Node:
    """
    A node class for A* Pathfinding
    """

    def __init__(self, parent=None, position=None, entry=None):
        self.parent = parent
        self.position = position
        self.id = entry

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return np.all(self.position == other.position)
    
    def __repr__(self):
        return f"{self.position} - g: {self.g} h: {self.h} f: {self.f}"

    # defining less than for purposes of heap queue
    def __lt__(self, other):
        if np.abs(self.f - other.f)<1e-6:
            return self.id < other.id
        else:
            return self.f < other.f
    
    # defining greater than for purposes of heap queue
    def __gt__(self, other):
        if np.abs(self.f - other.f)<1e-6:
            return self.id > other.id
        else:
            return self.f > other.f

def return_path(current_node):
    path = []
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    return path[::-1]  # Return reversed path


def astar(maze, start, end, allow_diagonal_movement = False, show_iter=[]):
    """
    Returns a list of tuples as a path from the given start to the given end in the given maze
    :param maze:
    :param start:
    :param end:
    :return:
    """
    global entry_id, ax, fig
    # Create start and end node
    start_node = Node(None, start, next(entry_id))
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end, next(entry_id))
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Heapify the open_list and Add the start node
    heapq.heapify(open_list) 
    heapq.heappush(open_list, start_node)

    # Adding a stop condition
    outer_iterations = 0
    max_iterations = 3*(len(maze[0]) * len(maze) // 4)

    # what squares do we search
    adjacent_squares = ((0, -1), (0, 1), (-1, 0), (1, 0),)
    if allow_diagonal_movement:
        adjacent_squares = ((0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1),)

    # Loop until you find the end
    while len(open_list) > 0:
         
        # Get the current node
        current_node = heapq.heappop(open_list)

        #if current_node in closed_list:
        #    print('Yipeeeeee!!!!!')
        #    continue
        
        outer_iterations += 1

        if outer_iterations > max_iterations:
          # if we hit this point return the path such as it is
          # it will not contain the destination
          warn("giving up on pathfinding too many iterations")
          return return_path(current_node)      
        #Show new nodes and save the figure
        if outer_iterations in show_iter or current_node == end_node or outer_iterations == max_iterations:
            #closed list
            closed_x = []
            closed_y = []
            closed_h = []
            for pt in closed_list:
                if pt!=start_node and pt!=end_node:
                    closed_x.append(pt.position[0])
                    closed_y.append(pt.position[1])
                    closed_h.append(pt.h)
            closed_x,closed_y = np.array([closed_x,closed_y])+1
            h_scatter = ax.scatter(closed_x,closed_y, 20, c=closed_h,cmap=plt.get_cmap(cmap_name),
                       vmin = 0, vmax = max(l,b) if allow_diagonal_movement else l+b, label = 'Heuristic')
            #open list
            open_x = []
            open_y = []
            open_h = []
            for pt in open_list:
                if pt!=start_node and pt!=end_node:
                    open_x.append(pt.position[0])
                    open_y.append(pt.position[1])
                    open_h.append(pt.h)
            open_x, open_y = np.array([open_x,open_y])+1
            ax.scatter(open_x, open_y, 20, color = 'lightblue', label = 'Open nodes')
            #save and pause
            fig.show()
            plt.savefig('iter'+str(outer_iterations)+cmap_name+'.png')
            plt.pause(1)


        # Found the goal
        if current_node == end_node:
            return return_path(current_node)

        # Generate children
        children = []
        
        for new_position in adjacent_squares: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0],node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position, next(entry_id))

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:
            
            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = max(np.abs(child.position[0]-end_node.position[0]),np.abs(child.position[1]-end_node.position[1])) if allow_diagonal_movement else np.abs(child.position[0]-end_node.position[0])+np.abs(child.position[1]-end_node.position[1])
            #returns estimated distance(ideal) required to travel
            #np.sqrt((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is on the closed list
            if len([closed_node for closed_node in closed_list if child == closed_node and child.g >= closed_node.g]) > 0:
                continue
            # Child is already in the open list
            if len([open_node for open_node in open_list if child == open_node and child.g >= open_node.g]) > 0:
                continue

            # Add the child to the open list
            heapq.heappush(open_list, child)


            
        closed_list.append(current_node)
    warn("Couldn't get a path to destination")
    return None

def example(print_maze = True):
    assert True == False
    maze = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] * 2,
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] * 2,
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] * 2,
            [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] * 2,
            [0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,] * 2,
            [0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,] * 2,
            [0,0,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,] * 2,
            [0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,1,0,] * 2,
            [0,0,0,1,0,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,] * 2,
            [0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,0,1,1,] * 2,
            [0,0,0,1,0,1,0,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,0,1,0,0,0,] * 2,
            [0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,] * 2,
            [0,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,0,0,] * 2,
            [0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,] * 2,
            [0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,] * 2,
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,] * 2,])
    
    start = (0, 0)
    end = (len(maze)-1, len(maze[0])-1)

    path = astar(maze, start, end)

    if print_maze:
      for step in path:
        maze[step[0]][step[1]] = 2
      
      for row in maze:
        line = []
        for col in row:
          if col == 1:
            line.append("\u2588")
          elif col == 0:
            line.append(" ")
          elif col == 2:
            line.append(".")
        print("".join(line))

    print(path)

## Run section 
Change options here.. Run the init() function to start

In [63]:
#add input here
l = 20
b = 30
allow_diagonal_movement = False
show_iter = [20, 50, 100, 150, 200,250]#saves figure on these iterations
#can add following through GUI. Either specify both here or specify both using GUI(i.e. mouse click)
src = None
dest = None
#maze_grid = None  ##makes own map if maze_grid is None
maze_grid = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] ,
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] ,
            [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] ,
            [0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,] ,
            [0,0,0,1,1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,] ,
            [0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,] ,
            [0,0,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,] ,
            [0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,1,0,] ,
            [0,0,0,1,0,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,] ,
            [0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,0,1,1,] ,
            [0,0,0,1,0,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,1,1,1,1,1,1,1,0,0,0,] ,
            [0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,1,1,0,] ,
            [0,0,0,1,0,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,1,1,0,0,0,] ,
            [0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,0,1,1,] ,
            [0,0,0,1,0,1,0,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,0,0,0,0,0,] ,
            [0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,] ,
            [0,0,0,1,0,1,1,1,1,0,1,0,0,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,0,0,] ,
            [0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,] ,
            [0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,] ,
            [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,] ,]) #should be 2-d np.ndarray
#l,b = np.shape(maze_grid)

In [64]:
cmap_name = 'cool' #change colour scheme here, matplotlib colors
obs_marker = 's'   #change obstacle marker to 'o' for reversible obstacle map

In [65]:
init()